# MLOps Workshop: Streamlit Integration

## Einführung
In diesem Notebook entwickeln wir eine interaktive Benutzeroberfläche mit Streamlit für unser Customer Churn Prediction Projekt. Wir werden verschiedene Visualisierungen erstellen und eine benutzerfreundliche Schnittstelle für Modellvorhersagen implementieren.

## Lernziele
Nach Abschluss dieses Notebooks können Sie:
- Eine Streamlit-App entwickeln
- Interaktive Visualisierungen erstellen
- Modellvorhersagen in Echtzeit durchführen
- Performance-Metriken visualisieren
- Ein Dashboard für Monitoring erstellen

## 1. Setup und Installation


In [ ]:
# Benötigte Bibliotheken importieren
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import requests
import mlflow
import json
from datetime import datetime, timedelta
import altair as alt

# MLflow Setup
mlflow.set_tracking_uri("file:./mlruns")



```python
```

## 2. Streamlit App Grundstruktur

### Aufgabe 1: Basis-App erstellen
Erstellen Sie die grundlegende Struktur der Streamlit-App.

```python
# app.py
import streamlit as st

def main():
    st.set_page_config(
        page_title="Churn Prediction Dashboard",
        page_icon="📊",
        layout="wide"
    )
    
    st.title("Customer Churn Prediction Dashboard")
    
    # Sidebar für Navigation
    page = st.sidebar.selectbox(
        "Navigation",
        ["Vorhersage", "Modell-Performance", "Monitoring"]
    )
    
    if page == "Vorhersage":
        show_prediction_page()
    elif page == "Modell-Performance":
        show_performance_page()
    else:
        show_monitoring_page()

if __name__ == "__main__":
    main()
```

## 3. Vorhersage-Interface

### Aufgabe 2: Vorhersage-Formular implementieren
Erstellen Sie ein interaktives Formular für Einzelvorhersagen.

```python
# prediction_page.py
def show_prediction_page():
    st.header("Churn-Vorhersage")
    
    with st.form("prediction_form"):
        col1, col2 = st.columns(2)
        
        with col1:
            gender = st.selectbox(
                "Geschlecht",
                ["Männlich", "Weiblich"]
            )
            
            tenure = st.number_input(
                "Vertragslaufzeit (Monate)",
                min_value=0,
                max_value=120
            )
            
            monthly_charges = st.number_input(
                "Monatliche Gebühren (€)",
                min_value=0.0
            )
        
        with col2:
            total_charges = st.number_input(
                "Gesamtgebühren (€)",
                min_value=0.0
            )
            
            avg_cost_service = st.number_input(
                "Durchschnittskosten pro Service (€)",
                min_value=0.0
            )
            
            customer_age = st.number_input(
                "Kundenalter (Jahre)",
                min_value=18,
                max_value=100
            )
        
        submitted = st.form_submit_button("Vorhersage starten")
        
        if submitted:
            # Daten für API-Anfrage vorbereiten
            data = {
                "gender": 1 if gender == "Weiblich" else 0,
                "tenure": tenure,
                "MonthlyCharges": monthly_charges,
                "TotalCharges": total_charges,
                "AvgCostPerService": avg_cost_service,
                "CustomerAge": customer_age
            }
            
            # API-Anfrage senden
            response = requests.post(
                "http://localhost:8000/predict",
                json={"data": data}
            )
            
            if response.status_code == 200:
                result = response.json()
                
                # Ergebnis visualisieren
                col1, col2 = st.columns(2)
                
                with col1:
                    if result["prediction"] == 1:
                        st.error("⚠️ Hohes Churn-Risiko!")
                    else:
                        st.success("✅ Geringes Churn-Risiko")
                
                with col2:
                    churn_prob = result["probability"] * 100
                    st.metric(
                        "Churn-Wahrscheinlichkeit",
                        f"{churn_prob:.1f}%"
                    )
                
                # Erklärungsgrafik
                fig = go.Figure(go.Indicator(
                    mode = "gauge+number",
                    value = churn_prob,
                    domain = {'x': [0, 1], 'y': [0, 1]},
                    title = {'text': "Churn-Risiko"},
                    gauge = {
                        'axis': {'range': [0, 100]},
                        'bar': {'color': "darkblue"},
                        'steps': [
                            {'range': [0, 30], 'color': "lightgreen"},
                            {'range': [30, 70], 'color': "yellow"},
                            {'range': [70, 100], 'color': "red"}
                        ]
                    }
                ))
                
                st.plotly_chart(fig)

```
